In [1]:
import os
work_dir = os.sep.join(os.getcwd().split(os.sep)[0:-1])

In [2]:
print(work_dir)

/Users/matthewlee/Documents/Code/b24/b5/bot2405a/dbc24/web/net/n5


In [3]:
import sys

work_tool=os.path.join(work_dir,'action')
data_dir_netflix=os.path.join(work_dir,'setup/d_data')
data_dir_edgar=os.path.join(os.sep.join(work_dir.split(os.sep)[0:-3]),'dat/net/sec_edgar')

sys.path.insert(0, work_tool)

In [4]:
from polyfuzz1 import polyfuzz2
from polyfuzz1 import metrics
from polyfuzz1.models import tfidf, BaseMatcher, EditDistance

from rapidfuzz import fuzz

import json
import numpy as np
import pandas as pd

## Data

In [5]:
# Load data
# data_a = json.load(open(f'{data_dir_netflix}/movie_titles_netflix_a.json', 'r'))
# data_b = json.load(open(f'{data_dir_netflix}/movie_titles_netflix_b.json', 'r'))

data_a = pd.read_csv(f'{data_dir_edgar}/edgar100a.csv')
data_b = pd.read_csv(f'{data_dir_edgar}/edgar100b.csv')
data_c = pd.read_csv(f'{data_dir_edgar}/edgar100c.csv')
data_d = pd.read_csv(f'{data_dir_edgar}/edgar100d.csv')

In [6]:
to_list = data_c.loc[:, 'name'].tolist()
from_list = data_b.loc[:, 'name'].tolist()

In [7]:
len(to_list)

100

## Custom Model/Grouping

In [8]:
class MyModel(BaseMatcher):
    def match(self, from_list, to_list, **kwargs):
        # Calculate distances
        matches = [[fuzz.ratio(from_string, to_string) / 100 
                   for to_string in to_list] for from_string in from_list]

        # Get best matches
        mappings = [to_list[index] for index in np.argmax(matches, axis=1)]
        scores = np.max(matches, axis=1)

        # Prepare dataframe
        matches = pd.DataFrame({'From': from_list,
                                'To': mappings, 
                                'Similarity': scores})
        return matches

In [9]:
# from_list = ["apple", "apples", "appl", "recal", "house", "similarity"]
# to_list = ["apple", "apples", "mouse"]

# model = PolyFuzz("TF-IDF").match(from_list, to_list)

# Custom grouper
base_edit_grouper = EditDistance(n_jobs=1)
# model.group(base_edit_grouper)

## Edit Distance

In [10]:
mymodel = MyModel()

In [11]:
model = polyfuzz2.PolyFuzzMatch(method=mymodel).match(to_list, from_list)

In [12]:
model.matches

,From,To,Similarity
0,1-800-PHARMACY INC,1-800-JACKPOT INC,0.685714
1,1-800-RADIATOR FRANCHISE INC.,1 800 RADIATOR FRANCHISE INC,0.912281
2,"1-900 JACKPOT, INC.",1-800-JACKPOT INC,0.833333
3,1-PAGE LTD,1-10/KATY LTD,0.608696
4,1/2 MAC LTD,1-10/KATY LTD,0.583333
...,...,...,...
95,"10101 SW 14 MEMBER, LLC","1 COMMUNITY BUILDERS, LLC",0.500000
96,"1010DATA, INC.","1-800-DOCTORS, INC.",0.666667
97,"1011 - 1041 TALEGA, LLC","1-800-PACK-RAT, LLC",0.476190
98,1011 BRIAR HILLS ONE INVESTORS INC,1 800 RADIATOR FRANCHISE INC,0.516129


In [13]:
grouper = polyfuzz2.PolyFuzzGroup(method=mymodel, matches=model.matches)

In [14]:
grouper.group()

In [16]:
grouper.matches

,From,To,Similarity,Group
0,1-800-PHARMACY INC,1-800-JACKPOT INC,0.685714,1-800-JACKPOT INC
1,1-800-RADIATOR FRANCHISE INC.,1 800 RADIATOR FRANCHISE INC,0.912281,1 800 RADIATOR FRANCHISE INC
2,"1-900 JACKPOT, INC.",1-800-JACKPOT INC,0.833333,1-800-JACKPOT INC
3,1-PAGE LTD,1-10/KATY LTD,0.608696,1-10/KATY LTD
4,1/2 MAC LTD,1-10/KATY LTD,0.583333,1-10/KATY LTD
...,...,...,...,...
95,"10101 SW 14 MEMBER, LLC","1 COMMUNITY BUILDERS, LLC",0.500000,"1 COMMUNITY BUILDERS, LLC"
96,"1010DATA, INC.","1-800-DOCTORS, INC.",0.666667,"1-800-DOCTORS, INC."
97,"1011 - 1041 TALEGA, LLC","1-800-PACK-RAT, LLC",0.476190,"1-800-PACK-RAT, LLC"
98,1011 BRIAR HILLS ONE INVESTORS INC,1 800 RADIATOR FRANCHISE INC,0.516129,1 800 RADIATOR FRANCHISE INC


## TF-IDF

model = polyfuzz1.PolyFuzz1("TF-IDF").match(to_list, from_list)
matches = model.matches['TF-IDF']

grouper = tfidf.TFIDF()
model.group(grouper)

model.matches['TF-IDF']

# model.method._extract_tf_idf(to_list, from_list)

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_a)
vectorizer.get_feature_names_out()[25:75]

vectorizer = CountVectorizer(min_df=1, analyzer=model.method._create_ngrams).fit(to_list + from_list)
tf_idf_to = vectorizer.transform(to_list)
tf_idf_from = vectorizer.transform(from_list)

print(np.array(tf_idf_to))

pd.DataFrame(metrics.precision_recall_curve(matches))

model.visualize_precision_recall()

model_a_to_b = polyfuzz1.Polyfuzz1